In [1]:
import joblib
import mlflow
import multiprocessing

from memoized_property import memoized_property
from mlflow.tracking import MlflowClient
from psutil import virtual_memory
from termcolor import colored
from time import time

from biodiversipy.params import MLFLOW_EXPERIMENT_BASE, MLFLOW_URI
from biodiversipy.utils import simple_time_tracker

In [2]:
# Create custom metric
def compute_average(y_true, y_pred, t):
    """Returns the average number of species observed correctly predicted given a threshold value t"""
    assert t <= 1
    assert t >= 0
    N, C = y_pred.shape
    temp = map(lambda x: 1 if x >= t else 0, y_pred[y_true == 1])

    average = reduce_sum(temp) / N
    return average

def find_t_min(y_true, y_pred, K, rate, t):
    """
    Returns the minimum threshold t and corresponding average satisfying the
    condition average <= K.
    The minimum t is found iteratively, with tuning parameter rate [0-1]
    """
    assert rate <= 1
    assert rate >= 0
    assert K > 0
    average = compute_average(y_true, y_pred, t)
    while average <= K:
        t = rate*t
        average = compute_average(y_true, y_pred, t)
    t_min = t/rate
    average = compute_average(y_true, y_pred, t_min)
    return t_min, average

def compute_accuracy(y_true, y_pred, t_min):
    N, C = y_pred.shape
    temp = map_fn(lambda x: 1 if x >= t_min else 0, y_pred[y_true == 1])

    return reduce_sum(temp) / (N*C)

def custom_metric(y_true, y_pred, K=2, rate=0.99, t=1):
    t_min, average = find_t_min(y_true, y_pred, K, rate, t)
    accuracy = compute_accuracy(y_true, y_pred, t_min)

    return t_min, average, accuracy

# Trying dummy model

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)

In [2]:
!pwd

/Users/mikkelvaldemarkoch/code/MikkelValdemar/biodiversipy/notebooks


In [3]:
X = pd.read_csv('../raw_data/output/occurrences/coordinates_100k/coordinates_100k_features.csv')
y = pd.read_csv('../raw_data/gbif/occurrences_100k/occurrences_100k_encoded.csv')
y = y.drop(columns=['latitude', 'longitude'])

In [12]:
X.head()

,latitude,longitude,bio_1,bio_2,bio_3,bio_4,bio_5,bio_6,bio_7,bio_8,...,silt_15-30cm,silt_30-60cm,silt_5-15cm,silt_60-100cm,soc_0-5cm,soc_100-200cm,soc_15-30cm,soc_30-60cm,soc_5-15cm,soc_60-100cm
0,48.870487,10.335366,8.025001,8.516667,32.382763,683.21265,22.7,-3.6,26.300001,14.950000,...,408,400,404,397,400,26,141,50,223,31
1,54.651142,9.772167,8.304167,6.225000,28.555044,589.65497,20.6,-1.2,21.800001,15.333334,...,209,206,218,200,727,278,461,385,545,283
2,53.936638,11.247078,8.629167,6.741666,29.311594,617.07690,21.5,-1.5,23.000000,16.366667,...,328,321,346,313,502,122,251,166,287,155
3,49.667747,8.651412,10.070833,8.091667,31.982872,660.06010,24.2,-1.1,25.300001,16.733334,...,532,516,556,515,480,29,158,71,246,32
4,48.826295,10.062125,8.412500,8.775000,33.492367,670.89197,23.0,-3.2,26.200000,15.250000,...,0,0,0,0,0,0,0,0,0,0


In [13]:
y.head()

,10009934,10010617,10055902,10068154,10096320,10220564,10243708,10356062,10448252,10461857,...,9573476,9606396,9624496,9643454,9677963,9689880,9823072,9823314,9823570,9868510
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
preproc_pipeline = make_pipeline(StandardScaler())
pipe_baseline = make_pipeline(preproc_pipeline, LogisticRegression())

In [4]:
X.describe()

,latitude,longitude,bio_1,bio_2,bio_3,bio_4,bio_5,bio_6,bio_7,bio_8,...,silt_15-30cm,silt_30-60cm,silt_5-15cm,silt_60-100cm,soc_0-5cm,soc_100-200cm,soc_15-30cm,soc_30-60cm,soc_5-15cm,soc_60-100cm
count,78274.000000,78274.000000,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,...,78274.000000,78274.000000,78274.000000,78274.000000,78274.000000,78274.000000,78274.000000,78274.000000,78274.000000,78274.000000
mean,50.821125,9.641268,-1.863454e+36,-1.863454e+36,-1.863454e+36,-1.863454e+36,-1.863454e+36,-1.863454e+36,-1.863454e+36,-1.863454e+36,...,257.675818,253.507614,266.377507,249.996819,454.005008,77.587500,171.718310,106.639919,241.058653,83.932838
std,1.754502,2.121061,2.510199e+37,2.510199e+37,2.510199e+37,2.510199e+37,2.510199e+37,2.510199e+37,2.510199e+37,2.510199e+37,...,216.379231,213.262630,225.013437,210.054376,389.949210,130.345536,181.993919,148.930762,228.498647,134.535184
min,47.278587,5.867479,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,49.460394,7.976818,8.475000e+00,7.616667e+00,3.135593e+01,6.012202e+02,2.210000e+01,-2.800000e+00,2.360000e+01,1.460000e+01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,50.928417,9.357782,9.075000e+00,8.066667e+00,3.244613e+01,6.451185e+02,2.300000e+01,-1.800000e+00,2.510000e+01,1.630000e+01,...,307.000000,300.000000,311.500000,297.500000,509.000000,38.000000,159.000000,72.000000,250.000000,45.000000
75%,52.111422,11.191132,9.616667e+00,8.475000e+00,3.336454e+01,6.741543e+02,2.380000e+01,-1.000000e+00,2.600000e+01,1.703333e+01,...,459.000000,453.000000,477.000000,444.000000,749.000000,96.000000,255.000000,144.000000,371.000000,106.000000
max,55.052055,14.998135,1.103333e+01,1.038333e+01,3.786438e+01,7.538140e+02,2.600000e+01,1.100000e+00,2.890000e+01,1.911667e+01,...,766.000000,719.000000,741.000000,717.000000,1752.000000,2345.000000,2389.000000,2373.000000,1770.000000,2313.000000


In [6]:
X[X['bio_1'] < -1000]

,latitude,longitude,bio_1,bio_2,bio_3,bio_4,bio_5,bio_6,bio_7,bio_8,...,silt_15-30cm,silt_30-60cm,silt_5-15cm,silt_60-100cm,soc_0-5cm,soc_100-200cm,soc_15-30cm,soc_30-60cm,soc_5-15cm,soc_60-100cm
75,54.260010,8.852663,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,...,0,0,0,0,0,0,0,0,0,0
384,54.398598,10.220761,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,...,0,0,0,0,0,0,0,0,0,0
588,54.338844,13.680669,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,...,0,0,0,0,0,0,0,0,0,0
788,54.402565,13.615970,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,...,0,0,0,0,0,0,0,0,0,0
1063,53.790783,7.901619,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77582,54.415115,10.232071,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,...,0,0,0,0,0,0,0,0,0,0
77708,54.134101,13.830972,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,...,0,0,0,0,0,0,0,0,0,0
77817,53.705471,7.137551,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,...,0,0,0,0,0,0,0,0,0,0
77854,54.763778,8.287171,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,...,251,245,247,250,956,587,613,720,560,642


In [ ]:

df_masked = np.ma.masked_where((-273 > df), df)

In [30]:
X_scaled = preproc_pipeline.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled.describe()

,latitude,longitude,bio_1,bio_2,bio_3,bio_4,bio_5,bio_6,bio_7,bio_8,...,silt_15-30cm,silt_30-60cm,silt_5-15cm,silt_60-100cm,soc_0-5cm,soc_100-200cm,soc_15-30cm,soc_30-60cm,soc_5-15cm,soc_60-100cm
count,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,...,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04,7.827400e+04
mean,1.269961e-15,-2.556262e-16,2.677902e-18,2.677902e-18,2.677902e-18,2.677902e-18,2.677902e-18,2.677902e-18,2.677902e-18,2.677902e-18,...,6.626673e-17,-4.883767e-17,-3.195327e-17,8.351423e-18,-7.879386e-17,2.015235e-17,1.879070e-17,-3.485812e-17,-1.343490e-17,-1.824604e-17
std,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,...,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00
min,-2.019126e+00,-1.779210e+00,-1.347059e+01,-1.347059e+01,-1.347059e+01,-1.347059e+01,-1.347059e+01,-1.347059e+01,-1.347059e+01,-1.347059e+01,...,-1.190860e+00,-1.188719e+00,-1.183837e+00,-1.190160e+00,-1.164274e+00,-5.952486e-01,-9.435448e-01,-7.160415e-01,-1.054974e+00,-6.238768e-01
25%,-7.755700e-01,-7.847300e-01,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,...,-1.190860e+00,-1.188719e+00,-1.183837e+00,-1.190160e+00,-1.164274e+00,-5.952486e-01,-9.435448e-01,-7.160415e-01,-1.054974e+00,-6.238768e-01
50%,6.115300e-02,-1.336538e-01,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,...,2.279539e-01,2.180067e-01,2.005337e-01,2.261485e-01,1.410321e-01,-3.037139e-01,-6.988361e-02,-2.325922e-01,3.913110e-02,-2.893896e-01
75%,7.354255e-01,7.307068e-01,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,...,9.304288e-01,9.354366e-01,9.360500e-01,9.235914e-01,7.565008e-01,1.412601e-01,4.576099e-01,2.508570e-01,5.686781e-01,1.640263e-01
max,2.411486e+00,2.525576e+00,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,7.423578e-02,...,2.349243e+00,2.182733e+00,2.109321e+00,2.223263e+00,3.328647e+00,1.739551e+01,1.218335e+01,1.521764e+01,6.691292e+00,1.656876e+01


In [ ]:
%%time
pipe_baseline.fit(X, y)

In [ ]:
cv_results = cross_validate(pipe_baseline, X=X, y=y, cv=5, scoring=rmsle)

In [ ]:
def model(X, y, metrics):
    normalization_layer = Normalization()
    normalization_layer.adapt(X)

    model = Sequential([
        normalization_layer,
        Dense(10,
              input_dim=X.shape[1],
              activation='relu',
              activity_regularizer=l1_l2(l1=0.005, l2=0.0005)),
        Dropout(0.4),
        Dense(y.shape[1], activation='softmax'),
    ])

    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    return model


In [7]:
y

,latitude,longitude,10009934,10010617,10055902,10068154,10096320,10220564,10243708,10356062,...,9573476,9606396,9624496,9643454,9677963,9689880,9823072,9823314,9823570,9868510
0,48.870487,10.335366,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,54.651142,9.772167,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,53.936638,11.247078,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,49.667747,8.651412,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,48.826295,10.062125,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78269,50.498398,6.624358,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
78270,53.321644,10.501041,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
78271,53.089754,10.565015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
78272,52.254784,12.928824,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
